In [8]:
## Delete existing seldon
!kind delete cluster --name seldon

Deleting cluster "seldon" ...


In [9]:
## Setup with K8S v1.21.2 to avoid apiextensions.k8s.io/v1beta1 CRD issue
## Refer https://stackoverflow.com/questions/69054622/unable-to-install-crds-in-kubernetes-kind
!kind create cluster --name seldon --image=kindest/node:v1.21.2

## Note: K8S >=1.25 has HPA(HorizontalPodAutoscaler) issues. So creating cluster with  kindest/node:v1.24.7 image.
# !kind create cluster --name seldon --image kindest/node:v1.24.7

Creating cluster "seldon" ...
 ✓ Ensuring node image (kindest/node:v1.21.2) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-seldon"
You can now use your cluster with:

kubectl cluster-info --context kind-seldon

Have a nice day! 👋


In [10]:
## set context
!kubectl cluster-info --context kind-seldon

Kubernetes control plane is running at https://127.0.0.1:57560
CoreDNS is running at https://127.0.0.1:57560/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [11]:
## Istio 
!istioctl install --set profile=default -y

✔ Istio core installed                                                          
✔ Istiod installed                                                              
✔ Ingress gateways installed                                                    
✔ Installation complete                                                         Making this installation the default for injection and validation.

Thank you for installing Istio 1.16.  Please take a few minutes to tell us about your install/upgrade experience!  https://forms.gle/99uiMML96AmsXY5d6


In [12]:
## istio-injection for K8S namespace default
!kubectl label namespace default istio-injection=enabled

namespace/default labeled


In [13]:
## Create Istio Gateway 
## Refer: https://docs.seldon.io/projects/seldon-core/en/latest/install/kind.html
!kubectl apply -f ./resources/create_istio_gateway.yaml

gateway.networking.istio.io/seldon-gateway created


In [14]:
### Install Seldon Core with istio.
# !kubectl delete namespace seldon-system
!kubectl create namespace seldon-system
!helm install seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts \
    --set usageMetrics.enabled=true \
    --set istio.enabled=true \
    --namespace seldon-system

namespace/seldon-system created
NAME: seldon-core
LAST DEPLOYED: Thu Dec  8 12:18:48 2022
NAMESPACE: seldon-system
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [20]:
### You can check that your Seldon Controller is running by doing:
!kubectl get pods -n seldon-system


NAME                                         READY   STATUS    RESTARTS   AGE
seldon-controller-manager-6d84cbd5d8-z6xkk   1/1     Running   0          115s


In [100]:
### Local Port Forwarding - run below command in a terminal
# !kubectl port-forward -n istio-system svc/istio-ingressgateway 8080:80 &